#Proyecto Chatbot Customer Service

In [ ]:
!pip install transformers
!pip install numpy
!pip install pandas
!pip install datasets
!pip install sklearn
!pip install torch
!pip install logging

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.1 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.0/96.0 kB 3.1 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subproc

In [ ]:
!pip install accelerate -U
!pip install transformers[torch] -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 5.9 MB/s eta 0:00:00


In [ ]:
!pip show accelerate

Name: accelerate
Version: 0.25.0
Summary: Accelerate
Home-page: https://github.com/huggingface/accelerate
Author: The HuggingFace team
Author-email: sylvain@huggingface.co
License: Apache
Location: /usr/local/lib/python3.10/dist-packages
Requires: huggingface-hub, numpy, packaging, psutil, pyyaml, safetensors, torch
Required-by: 


In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
from transformers import TrainingArguments
import torch

Revisar cuda

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


Load dataset from a csv or directly from hugginface

In [ ]:
# Load your dataset
dataset = pd.read_csv('converted.csv')
dataset.head()


,issue_area,issue_category,issue_sub_category,issue_category_sub_category,customer_sentiment,product_category,product_sub_category,issue_complexity,agent_experience_level,agent_experience_level_desc,conversation
0,Login and Account,Mobile Number and Email Verification,Verification requirement for mobile number or ...,Mobile Number and Email Verification -> Verifi...,neutral,Appliances,Oven Toaster Grills (OTG),medium,junior,"handles customer inquiries independently, poss...",Agent: Thank you for calling BrownBox Customer...
1,Cancellations and returns,Pickup and Shipping,Reasons for being asked to ship the item,Pickup and Shipping -> Reasons for being asked...,neutral,Electronics,Computer Monitor,less,junior,"handles customer inquiries independently, poss...",Agent: Thank you for calling BrownBox customer...
2,Cancellations and returns,Replacement and Return Process,Inability to click the 'Cancel' button,Replacement and Return Process -> Inability to...,neutral,Appliances,Juicer/Mixer/Grinder,medium,experienced,"confidently handles complex customer issues, e...",Agent: Thank you for calling BrownBox Customer...
3,Login and Account,Login Issues and Error Messages,Error message regarding exceeded attempts to e...,Login Issues and Error Messages -> Error messa...,neutral,Appliances,Water Purifier,less,inexperienced,"may struggle with ambiguous queries, rely on c...","Customer: Hi, I am facing an issue while loggi..."
4,Order,Order Delivery Issues,Delivery not attempted again,Order Delivery Issues -> Delivery not attempte...,negative,Electronics,Bp Monitor,medium,experienced,"confidently handles complex customer issues, e...",Agent: Thank you for contacting BrownBox custo...


In [ ]:
from sklearn.model_selection import train_test_split

# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained("Kaludi/Customer-Support-Assistant-V2")

# Extract the 'conversation' column
conversations = dataset['conversation']

# Basic preprocessing (optional based on your requirement)
conversations = conversations.str.lower()
#conversations = conversations.str.replace(r'[^\w\s]', '')
conversations.dropna(inplace=True)

# Split the preprocessed conversations into training and test sets
train_conversations, test_conversations = train_test_split(conversations, test_size=0.2, random_state=42)



# Tokenize the training data
tokenized_train_data = tokenizer(
    train_conversations.tolist(),
    padding="max_length",
    truncation=True,
    max_length=512,
    return_tensors="pt"
)

# Tokenize the test data
tokenized_test_data = tokenizer(
    test_conversations.tolist(),
    padding="max_length",
    truncation=True,
    max_length=512,
    return_tensors="pt"
)


In [ ]:
conversations.head()

0    agent: thank you for calling brownbox customer...
1    agent: thank you for calling brownbox customer...
2    agent: thank you for calling brownbox customer...
3    customer: hi, i am facing an issue while loggi...
4    agent: thank you for contacting brownbox custo...
Name: conversation, dtype: object

Función para separar la conversación entre inputs y respuestas target

In [ ]:
def parse_conversation(conversations):
    customer_inputs = []
    agent_responses = []
    for conversation in conversations:
        turns = conversation.split('\n')
        for i in range(len(turns)):
            if turns[i].startswith("customer:"):
                # Extract customer message, removing the prefix and leading/trailing whitespace
                customer_msg = turns[i].split("customer: ", 1)[1].strip()
                # Initialize agent message as empty
                agent_msg = ""
                # Look ahead for the next agent message
                for j in range(i+1, len(turns)):
                    if turns[j].startswith("agent:"):
                        agent_msg = turns[j].split("agent: ", 1)[1].strip()
                        break  # Stop at the first agent response after the customer message
                customer_inputs.append(customer_msg)
                agent_responses.append(agent_msg)
    return customer_inputs, agent_responses


In [ ]:
train_customer_inputs, train_agent_responses = parse_conversation(train_conversations)
test_customer_inputs, test_agent_responses = parse_conversation(test_conversations)


In [ ]:
train_customer_inputs[:5]


["hi sarah, i'm calling to change the address for the item pick-up of my pram/stroller.",
 'my order number is 789012 and i purchased it on the 15th of this month.',
 'yes, the new address is 123 main street, anytown, usa.',
 "no, it's not within the same zip code.",
 "this is ridiculous. i don't want to cancel the order. i need the pram/stroller, and i cannot wait for a new one to arrive."]

In [ ]:
train_agent_responses[:10]

["i'm sorry to hear that you need to change the address. i'll be happy to assist you with that. can you please provide me with your order number and the date of purchase?",
 'thank you for the information. i understand that you want to change the address for the item pick-up. can you please confirm the new address?',
 "thank you for providing the new address. let me check if it's possible to change the address for the item pick-up. please hold on for a moment.",
 "i'm sorry, but we cannot change the address for the item pick-up if it's not within the same zip code. you can cancel the order and place a new one with the correct address.",
 'i completely understand your situation. let me check if there are any other options available. please hold on for a moment.',
 "you're welcome. let me make the necessary arrangements. please hold on for a moment.",
 "you're welcome. is there anything else i can help you with?",
 "you're welcome. have a great day!",
 "hello, i'm sorry to hear that you'

In [ ]:
# Tokenize training data
tokenized_train_inputs = tokenizer(train_customer_inputs, padding=True, truncation=True, max_length=512, return_tensors="pt")
tokenized_train_labels = tokenizer(train_agent_responses, padding=True, truncation=True, max_length=512, return_tensors="pt")

# Tokenize test data
tokenized_test_inputs = tokenizer(test_customer_inputs, padding=True, truncation=True, max_length=512, return_tensors="pt")
tokenized_test_labels = tokenizer(test_agent_responses, padding=True, truncation=True, max_length=512, return_tensors="pt")


In [ ]:
class ConversationDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels['input_ids'][idx]
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

In [ ]:
#Datasets Instances
train_dataset = ConversationDataset(tokenized_train_inputs, tokenized_train_labels)
test_dataset = ConversationDataset(tokenized_test_inputs, tokenized_test_labels)


In [ ]:
train_dataset[0]

{'input_ids': tensor([    0,  3592,   579, 36000,     6,   939,   437,  1765,     7,   464,
             5,  1100,    13,     5,  6880,  1339,    12,   658,     9,   127,
          3349,   424,    73,   620, 32121,     4,     2,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'labels': tensor([   0,  118,  437, 6661,    7, 1798,   14,   47,  240,    7,  464,    5,
         1100,    4,  939,  581,   28, 1372,    7, 3991,   47,   19,   14,    4,
           64

In [ ]:
test_dataset[0]

{'input_ids': tensor([    0,  3592,   579, 36000,     6,   939,   524,   667,     7,   120,
            11,  2842,    19,     5, 37171,   544,  3696,  3059,    19,     5,
         15689,     9,    10,  3349,   424,    73,   620, 32121,    14,   939,
            33,  2162,    31,  6219,  8304,     4,    64,    47,   244,   162,
            19,    14,   116,     2,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'labels': tensor([    0, 41404,     6,   939,  1017,    28,  1372,     7,  3991,    47,
            19,    14,     4,   189,   939,    33,   110,   645,   346,     6,
          2540,   116,     2,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,    

Load Pretrained Model

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("Kaludi/Customer-Support-Assistant-V2", from_tf=True)


config.json:   0%|          | 0.00/1.64k [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

All TF 2.0 model weights were used when initializing BartForConditionalGeneration.

Some weights of BartForConditionalGeneration were not initialized from the TF 2.0 model and are newly initialized: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/358 [00:00<?, ?B/s]

In [ ]:
import logging

logging.basicConfig(level=logging.INFO)


In [ ]:
from transformers import TrainingArguments

# Define your training arguments
training_args = TrainingArguments(
    output_dir="results",
    num_train_epochs=5,
    per_device_train_batch_size=12,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
)

# Initialize the Trainer
trainer = Trainer(
    model=model.to(device),
    args=training_args,
    train_dataset=train_dataset
)


In [ ]:
from sklearn.metrics import accuracy_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc}

# Add this function to the Trainer
trainer = Trainer(
    model=model.to(device),
    args=training_args,
    train_dataset=train_dataset,
    compute_metrics=compute_metrics,
    )

In [ ]:
# Fine-tune the model
trainer.train()

Step,Training Loss
500,0.094600
1000,0.111000
1500,0.114200
2000,0.080700
2500,0.056300


TrainOutput(global_step=2725, training_loss=0.08767336329188916, metrics={'train_runtime': 821.8728, 'train_samples_per_second': 39.751, 'train_steps_per_second': 3.316, 'total_flos': 4770656452362240.0, 'train_loss': 0.08767336329188916, 'epoch': 5.0})

Save model in Huggin Face

In [ ]:
!pip install transformers huggingface_hub

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' creden

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
model.save_pretrained('results/model1')

In [ ]:
! ls results

checkpoint-1000  checkpoint-1500  checkpoint-2000  checkpoint-2500  checkpoint-500  model1


In [ ]:
from huggingface_hub import HfApi
api = HfApi()

# Modificalo para que sea tu repositorio
api.upload_folder(
    folder_path="results/model1", # Modelo a guardar
    repo_id="SantiagoPG/chatbot_customer_service", # Repositorio
    repo_type="model", # Tipo de repositorio, en nuestro caso modelo
    #path_in_repo="model1", # Carpeta para guardar el modelo

)

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

'https://huggingface.co/SantiagoPG/chatbot_customer_service/tree/main/'

In [ ]:

# Load the model for inference
model = AutoModelForSeq2SeqLM.from_pretrained('SantiagoPG/chatbot_customer_service')
tokenizer = AutoTokenizer.from_pretrained("Kaludi/Customer-Support-Assistant-V2")

config.json:   0%|          | 0.00/1.67k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/358 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [ ]:
import torch

def generate_response(input_text):
    # Asegúrate de que el modelo y el tokenizador estén en el dispositivo correcto (GPU o CPU)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Tokeniza el texto de entrada: convierte el texto en un formato que el modelo pueda entender
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

    # Genera una respuesta usando el modelo
    output = model.generate(input_ids)

    # Decodifica la respuesta generada de nuevo a texto legible por humanos
    response = tokenizer.decode(output[0], skip_special_tokens=True)

    return response



In [ ]:
#Conversación de Prueba

user_input = "hi!"
response = generate_response(user_input)
print("Chatbot:", response)


Chatbot: hello! thank you for contacting brownbox customer support. my name is sarah, and i'll be happy to assist you. may i have your name and email address, please, so i can keep you updated on the latest updates on the availability of our products and services?


In [ ]:
#Interacción Continua:
while True:
    user_input = input("You: ")
    if user_input.lower() == 'quit':
        break
    response = generate_response(user_input)
    print("Chatbot:", response)


Chatbot: hello! thank you for contacting brownbox customer support. my name is sarah, and i'll be happy to assist you. may i have your name and email address, please, so i can keep you updated on the latest updates on the availability of our products and services?
